<a href="https://colab.research.google.com/github/jamg-upv/Impacto-citas-articulo/blob/main/impactoencitas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

How to cite this work:

------------

Este codigo permite integrar una tabla con los datos de un conjunto de articulos y sus numeros de citas, normalizarlas teniendo en cuenta las tablas de citas mundiales (scopus) o baseline de (WOS), y añadir las métricas de impacto de la revista en el año de publicación del articulo.
Son necesarias 3 tablas para cada proveedor (scopus y WOS)


Fuente de datos SCOPUS https://elsevier.com/?a=734751


Tareas 
1. Lista de 25 DOIs Banco de Pruebas1  (meritos (5 autores "al azar") )
2. Descargar como cvs bancopruebas1sco tabla Scopus  de 25 meritos (5 autores "al azar") a partir de los DOI
1. Descargar como cvs bancopruebas1wos tabla WOS de los 25 meritos a partir de los DOI
1. preparar un prototipo de la tabla que penamos crear para consultar a compañeros del equipo
2.   



Pasos previos. Librerias y configuración

In [5]:
# Importamos ls librería de Python que necesitaremos en este notebook

import pandas as pd  # Pandas permite manipular tablas 
import matplotlib.pyplot as plt  # Para hacer gráficas
import seaborn as sns  # Para hacer gráficas pero necesita matplotlib y deja unos graficos chulisismos 
sns.set_style('whitegrid')  # Pone una malla de fondo bastante mona para Seaborn

import os #rename files
#import xlwings as xw #permite leer docuemtnos xlb me da error y no lo puedo instalar

# Redondeamos las salidas a dos decimales
%precision %.2f

# from google.colab import drive
# drive.mount('/content/drive')



'%.2f'

Carga de *datos* Indicadores de revistas SCOPUS

In [6]:
#métricas de revistas Scopus
# si lo descargamos directo de la web Ç(ventaja es que con una descarga siempre está accesible y no hay que volver a descargarlo durante la sesión)
# el archivo de SCopus de 10 años ocupa demasiados GB (60gb) y no se puede alojar en drive (no debería esar en un acceso libre en github porque es propiedad de Scopus)
   #!wget https://elsevier.com/?a=734751 
#es XLB, hay que convertirlo a mano XLS porque xlb requiere de librerias que no tengo instaladas en colab
# y hacerlo accesible aquí
#la primer pesaña son definiciones de indicadores, la segunra el el año más reciente y luego 9 mas con los anteriores. La ultima pestaña es un clasificación de categorias
## limpio a mano las pestañas no usadas (priemera y ultima) y dejo solo los 7 años más recientes (para que sean solo 44mb y me quepa en google drive)
!wget --output-document=CiteScore14_20.xlsx https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/egXj6G-GT0WY_g3O-2_swA
#si lo hago manual, que tarda una eternidad cada vez)

# en general funciona mejor con archivos csv, que con excel. tendre que partir el archivo excel en csv

#renbombrar el fichero local descargado para que no sea tan feote
#os.rename(r'/content/edit?usp=sharing',r'/content/scopus.xlsx')

--2022-02-08 20:07:51--  https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/egXj6G-GT0WY_g3O-2_swA
Resolving gdocu.upv.es (gdocu.upv.es)... 158.42.4.31
Connecting to gdocu.upv.es (gdocu.upv.es)|158.42.4.31|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 62207801 (59M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘CS14_20.xlsx’

CS14_20.xlsx        100%[===================>]  59.33M  18.0MB/s    in 3.3s    

2022-02-08 20:07:55 (18.0 MB/s) - ‘CS14_20.xlsx’ saved [62207801/62207801]



In [ ]:
# si sibo el fichero a mano a googlecolab (varios minutos en subir y 130seg en cargar el dataframe)

dfs = pd.read_excel('/content/CiteScore14_20.xlsx',engine='openpyxl',sheet_name=None)
dfs

In [ ]:
# Otra forma de hacer exactamente lo mismo de arriba (130 segundo)

xl_file = pd.ExcelFile('/content/CiteScore14_20.xlsx')

dfs2 = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}
dfs2

In [ ]:
##pendiente automtizar la creación de un CSV a partir de las pestañas, añadiendo una columna del año (y rotulando citescore solo sin años en las columnas de datos)

Preparacion de las pestañas d la excel
Quitar espacioes en los rotulos del encabezado
Estandarizar los nombres de las columnas

ScopusSourceID	Title	CitationCount	ScholarlyOutput	PercentCited	CiteScore	SNIP	SJR	ScopusASJCCode	ScopusSubSubjectArea	Percentile	RANK	RankOutOf	Publisher	Type	OpenAccess	Quartile	Top10	URLScopusSourceID	PrintISSN	ElectISSN


In [3]:
# importar tablas de metricas revisas SCOPUS (la excel original la he convertido en un CVS por cada pestaña)
# hay que importar los csv y ñadir a cada uno un campo con el año y luego fusionarlos en una tabla unica o trabajarlos como tablas separadas (no sé que será más eficiente)
# El csv subido a Github no funciona (crea tags de html en el archivo y no se lee luego con pandas. Lo lee pero los nombres de los ficheros no son intuitivos)
# !wget https://github.com/jamg-upv/Impacto-citas-articulo/blob/main/tablasinput/CiteScore2020.csv


--2022-02-08 20:00:51--  https://github.com/jamg-upv/Impacto-citas-articulo/blob/main/tablasinput/CiteScore2020.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘CiteScore2020.csv’

CiteScore2020.csv       [ <=>                ] 169.11K  --.-KB/s    in 0.04s   

2022-02-08 20:00:51 (4.28 MB/s) - ‘CiteScore2020.csv’ saved [173167]



In [7]:
#lecturas en orden inverso de 2020 hasta 2011
!wget --output-document=CS2020.cvs  https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/bCd_63LrSVSkf96qwXihVQ  #2020
!wget --output-document=CS2019.cvs  https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/5XemCCZCS-WNmJ-byA6uzA #2019
!wget --output-document=CS2018.cvs  https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/H3ZrGBF1TNuBDCI95AVHkA #2018
!wget --output-document=CS2017.cvs  https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/kTZ3Mlj8Qr6DC08kauJ2og #2017
!wget --output-document=CS2016.cvs  https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/N3l6wTevTLeD6fgI42Hlvw #2016
!wget --output-document=CS2015.cvs  https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/oYOpp1OcTIKXBzXfsDuf2g  #2015
!wget --output-document=CS2014.cvs  https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/FMHpXVs4SXygTdZGYVp1kA #2014
!wget --output-document=CS2013.cvs  https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/XOSnXVhMSFG3lC3hg8bHmg #2013
!wget --output-document=CS2012.cvs  https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/IE_0VEppTFOxR2MEwMy9Nw #2012
!wget --output-document=CS2011.cvs  https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/Q_O9A4pMTmaXZmxdEaMmSw #2011

--2022-02-08 20:14:44--  https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/bCd_63LrSVSkf96qwXihVQ
Resolving gdocu.upv.es (gdocu.upv.es)... 158.42.4.31
Connecting to gdocu.upv.es (gdocu.upv.es)|158.42.4.31|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 11980999 (11M) [text/csv]
Saving to: ‘CS2020.cvs’

CS2020.cvs          100%[===================>]  11.43M  8.55MB/s    in 1.3s    

2022-02-08 20:14:46 (8.55 MB/s) - ‘CS2020.cvs’ saved [11980999/11980999]

--2022-02-08 20:14:46--  https://gdocu.upv.es/share/proxy/alfresco-noauth/api/internal/shared/node/content/5XemCCZCS-WNmJ-byA6uzA
Resolving gdocu.upv.es (gdocu.upv.es)... 158.42.4.31
Connecting to gdocu.upv.es (gdocu.upv.es)|158.42.4.31|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 11514712 (11M) [text/csv]
Saving to: ‘CS2019.cvs’

CS2019.cvs          100%[===================>]  10.98M  9.77MB/s    in 1.1s    

2022-02-08 20:14:48 (9.77 MB/s) - ‘CS2019.cv

In [8]:
CS2020= pd.read_csv('CS2020.cvs',sep=';')
CS2019= pd.read_csv('CS2019.cvs',sep=';')
CS2018= pd.read_csv('CS2018.cvs',sep=';')
CS2017= pd.read_csv('CS2017.cvs',sep=';')
CS2016= pd.read_csv('CS2016.cvs',sep=';')
CS2015= pd.read_csv('CS2015.cvs',sep=';')
CS2014= pd.read_csv('CS2014.cvs',sep=';')
CS2013= pd.read_csv('CS2013.cvs',sep=';')
CS2012= pd.read_csv('CS2012.cvs',sep=';')
CS2011= pd.read_csv('CS2011.cvs',sep=';')


In [9]:
CS2020


,ScopusSourceID,Title,CitationCount,ScholarlyOutput,PercentCited,CiteScore,SNIP,SJR,ScopusASJCCode,ScopusSubSubjectArea,Percentile,RANK,RankOutOf,Publisher,Type,OpenAccess,Quartile,Top10,URLScopusSourceID,PrintISSN,ElectISSN
0,21101021140,1700-tal: Nordic Journal for Eighteenth-Centur...,0,5,0,0,0,0,1213,Visual Arts and Performing Arts,6,466,532,Swedish Society for Eighteenth-Century Studies,j,YES,4,FALSO,https://www.scopus.com/sourceid/21101021140,NaN,NaN
1,21101021140,1700-tal: Nordic Journal for Eighteenth-Centur...,0,5,0,0,0,0,1202,History,3,1235,1328,Swedish Society for Eighteenth-Century Studies,j,YES,4,FALSO,https://www.scopus.com/sourceid/21101021140,NaN,NaN
2,21101021140,1700-tal: Nordic Journal for Eighteenth-Centur...,0,5,0,0,0,0,3316,Cultural Studies,2,986,1037,Swedish Society for Eighteenth-Century Studies,j,YES,4,FALSO,https://www.scopus.com/sourceid/21101021140,NaN,NaN
3,21100404576,2D Materials,12778,920,90,"13,9","1,402","2,702",2210,Mechanical Engineering,97,14,596,Institute of Physics Publishing,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/21100404576,NaN,20531583
4,21100404576,2D Materials,12778,920,90,"13,9","1,402","2,702",2211,Mechanics of Materials,97,11,377,Institute of Physics Publishing,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/21100404576,NaN,20531583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59339,28883,Zygon,301,237,56,"1,3","0,853","0,222",3304,Education,46,707,1319,Wiley-Blackwell,j,NO,3,FALSO,https://www.scopus.com/sourceid/28883,5912385,14679744
59340,19073,Zygote,588,206,69,"2,9","0,559","0,416",1309,Developmental Biology,25,61,81,Cambridge University Press,j,NO,3,FALSO,https://www.scopus.com/sourceid/19073,9671994,14698730
59341,19073,Zygote,588,206,69,"2,9","0,559","0,416",1307,Cell Biology,22,218,279,Cambridge University Press,j,NO,4,FALSO,https://www.scopus.com/sourceid/19073,9671994,14698730
59342,19700173167,Zywnosc. Nauka. Technologia. Jakosc/Food. Scie...,111,191,32,"0,6","0,279","0,149",2209,Industrial and Manufacturing Engineering,23,259,336,Polish Society of Food Technologists,j,NO,4,FALSO,https://www.scopus.com/sourceid/19700173167,14256959,NaN


In [ ]:
CS2019


,ScopusSourceID,Title,CitationCount,ScholarlyOutput,PercentCited,CiteScore,SNIP,SJR,ScopusASJCCode,ScopusSubSubjectArea,Percentile,RANK,RankOutOf,Publisher,Type,OpenAccess,Quartile,Top10,URLScopusSourceID,PrintISSN,ElectISSN
0,12128,Behavioural and Cognitive Psychotherapy,652,226,"70,8","2,9","0,981","0,805",3203.0,Clinical Psychology,68.0,88.0,275.0,Cambridge University Press,j,NO,2.0,FALSO,https://www.scopus.com/sourceid/12128,13524658,14691833
1,12366,Photogrammetric Engineering and Remote Sensing,644,260,"60,8","2,5","0,708","0,521",1903.0,Computers in Earth Sciences,48.0,21.0,40.0,American Society for Photogrammetry and Remote...,j,NO,3.0,FALSO,https://www.scopus.com/sourceid/12366,991112,NaN
2,12894,Chinese Journal of Chromatography (Se Pu),649,731,"44,5","0,9","0,282","0,199",1500.0,General Chemical Engineering,29.0,200.0,281.0,Zhongguo Kexueyan Ganguang Huaxue Yanjiusuo,j,NO,3.0,FALSO,https://www.scopus.com/sourceid/12894,10008713,18722059
3,12894,Chinese Journal of Chromatography (Se Pu),649,731,"44,5","0,9","0,282","0,199",1603.0,Electrochemistry,17.0,31.0,37.0,Zhongguo Kexueyan Ganguang Huaxue Yanjiusuo,j,NO,4.0,FALSO,https://www.scopus.com/sourceid/12894,10008713,18722059
4,12894,Chinese Journal of Chromatography (Se Pu),649,731,"44,5","0,9","0,282","0,199",1602.0,Analytical Chemistry,15.0,101.0,119.0,Zhongguo Kexueyan Ganguang Huaxue Yanjiusuo,j,NO,4.0,FALSO,https://www.scopus.com/sourceid/12894,10008713,18722059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57013,21100925891,Journal of Early Modern Studies,0,7,0,0,NaN,NaN,3310.0,Linguistics and Language,3.0,820.0,884.0,Firenze University Press,j,NO,4.0,FALSO,https://www.scopus.com/sourceid/21100925891,22797149,NaN
57014,21100925891,Journal of Early Modern Studies,0,7,0,0,NaN,NaN,1208.0,Literature and Literary Theory,5.0,727.0,823.0,Firenze University Press,j,NO,4.0,FALSO,https://www.scopus.com/sourceid/21100925891,22797149,NaN
57015,21100925891,Journal of Early Modern Studies,0,7,0,0,NaN,NaN,1202.0,History,4.0,1158.0,1259.0,Firenze University Press,j,NO,4.0,FALSO,https://www.scopus.com/sourceid/21100925891,22797149,NaN
57016,21100937999,Australian Journal of Cancer Nursing,1,7,"14,3","0,1",NaN,NaN,2917.0,Oncology (nursing),2.0,17.0,17.0,Cambridge Media,j,NO,4.0,FALSO,https://www.scopus.com/sourceid/21100937999,14412551,NaN


In [ ]:
CS2018


,ScopusSourceID,Title,CitationCount,ScholarlyOutput,PercentCited,CiteScore,SNIP,SJR,ScopusASJCCode,ScopusSubSubjectArea,Percentile,RANK,RankOutOf,Publisher,Type,OpenAccess,Quartile,Top10,URLScopusSourceID,PrintISSN,ElectISSN
0,12100155657,Mundo Agrario,40,124,"19,4","0,3","0,558","0,149",3305.0,"Geography, Planning and Development",18.0,537.0,660.0,Universidad Nacional de La Plata,j,YES,4.0,FALSO,https://www.scopus.com/sourceid/12100155657,15155994,NaN
1,12100155657,Mundo Agrario,40,124,"19,4","0,3","0,558","0,149",1410.0,Industrial Relations,20.0,46.0,57.0,Universidad Nacional de La Plata,j,YES,4.0,FALSO,https://www.scopus.com/sourceid/12100155657,15155994,NaN
2,12100155657,Mundo Agrario,40,124,"19,4","0,3","0,558","0,149",1202.0,History,55.0,542.0,1227.0,Universidad Nacional de La Plata,j,YES,2.0,FALSO,https://www.scopus.com/sourceid/12100155657,15155994,NaN
3,12100155657,Mundo Agrario,40,124,"19,4","0,3","0,558","0,149",3322.0,Urban Studies,33.0,122.0,183.0,Universidad Nacional de La Plata,j,YES,3.0,FALSO,https://www.scopus.com/sourceid/12100155657,15155994,NaN
4,12100155657,Mundo Agrario,40,124,"19,4","0,3","0,558","0,149",1101.0,Agricultural and Biological Sciences (miscella...,15.0,56.0,66.0,Universidad Nacional de La Plata,j,YES,4.0,FALSO,https://www.scopus.com/sourceid/12100155657,15155994,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55082,21100865931,EC Tax Review,4,29,"10,3","0,1",NaN,NaN,1402.0,Accounting,7.0,139.0,149.0,Springer Nature,j,NO,4.0,FALSO,https://www.scopus.com/sourceid/21100865931,9282750,18758363
55083,21100887653,African Vision and Eye Health,71,113,"30,1","0,6","0,459",NaN,3610.0,Optometry,18.0,7.0,8.0,OpenJournals Publishing AOSIS (Pty) Ltd,j,YES,4.0,FALSO,https://www.scopus.com/sourceid/21100887653,24133183,24101516
55084,21100887653,African Vision and Eye Health,71,113,"30,1","0,6","0,459",NaN,2731.0,Ophthalmology,23.0,88.0,114.0,OpenJournals Publishing AOSIS (Pty) Ltd,j,YES,4.0,FALSO,https://www.scopus.com/sourceid/21100887653,24133183,24101516
55085,21100902964,Applied Mathematics and Mechanics,124,305,"21,3","0,4","0,39",NaN,2210.0,Mechanical Engineering,16.0,491.0,587.0,Editorial Office of Applied Mathematics and Me...,j,NO,4.0,FALSO,https://www.scopus.com/sourceid/21100902964,10000887,NaN


In [ ]:
CS2017


,ScopusSourceID,Title,CitationCount,ScholarlyOutput,PercentCited,CiteScore,SNIP,SJR,ScopusASJCCode,ScopusSubSubjectArea,Percentile,RANK,RankOutOf,Publisher,Type,OpenAccess,Quartile,Top10,URLScopusSourceID,PrintISSN,ElectISSN
0,21100837274,Physical Therapy Reviews,85,101,"34,7","0,8","0,388","0,28",3612,"Physical Therapy, Sports Therapy and Rehabilit...",35,114,176,Taylor & Francis,j,NO,3,FALSO,https://www.scopus.com/sourceid/21100837274,10833196,1743288X
1,21100837274,Physical Therapy Reviews,85,101,"34,7","0,8","0,388","0,28",2732,Orthopedics and Sports Medicine,26,180,245,Taylor & Francis,j,NO,3,FALSO,https://www.scopus.com/sourceid/21100837274,10833196,1743288X
2,21100837274,Physical Therapy Reviews,85,101,"34,7","0,8","0,388","0,28",2742,Rehabilitation,32,78,114,Taylor & Francis,j,NO,3,FALSO,https://www.scopus.com/sourceid/21100837274,10833196,1743288X
3,21100838044,"Geopolitics, History, and International Relations",1,11,"9,1","0,1",NaN,NaN,1202,History,22,843,1090,Addleton Academic Publishers,j,NO,4,FALSO,https://www.scopus.com/sourceid/21100838044,19489145,23744383
4,21100838044,"Geopolitics, History, and International Relations",1,11,"9,1","0,1",NaN,NaN,3320,Political Science and International Relations,8,420,458,Addleton Academic Publishers,j,NO,4,FALSO,https://www.scopus.com/sourceid/21100838044,19489145,23744383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52780,21100826771,Springer Tracts in Mechanical Engineering,75,194,"19,1","0,4","2,034","0,12",2202,Aerospace Engineering,18,97,119,Springer Nature,k,NO,4,FALSO,https://www.scopus.com/sourceid/21100826771,21959862,21959870
52781,21100826771,Springer Tracts in Mechanical Engineering,75,194,"19,1","0,4","2,034","0,12",2207,Control and Systems Engineering,12,207,236,Springer Nature,k,NO,4,FALSO,https://www.scopus.com/sourceid/21100826771,21959862,21959870
52782,21100833806,Journal of the Korean Society of Clothing and ...,0,85,0,0,NaN,NaN,2209,Industrial and Manufacturing Engineering,3,304,328,Korean Society of Clothing and Textiles,j,NO,4,FALSO,https://www.scopus.com/sourceid/21100833806,12251151,22340793
52783,21100833806,Journal of the Korean Society of Clothing and ...,0,85,0,0,NaN,NaN,2507,Polymers and Plastics,2,142,147,Korean Society of Clothing and Textiles,j,NO,4,FALSO,https://www.scopus.com/sourceid/21100833806,12251151,22340793


In [ ]:
CS2016


,ScopusSourceID,Title,CitationCount,ScholarlyOutput,PercentCited,CiteScore,SNIP,SJR,ScopusASJCCode,ScopusSubSubjectArea,Percentile,RANK,RankOutOf,Publisher,Type,OpenAccess,Quartile,Top10,URLScopusSourceID,PrintISSN,ElectISSN
0,12167,Marine Ecology,835,354,"70,9","2,4","0,736","0,64",1104,Aquatic Science,57,86,202,Wiley-Blackwell,j,NO,2,FALSO,https://www.scopus.com/sourceid/12167,1739565,14390485
1,12167,Marine Ecology,835,354,"70,9","2,4","0,736","0,64",2303,Ecology,58,128,310,Wiley-Blackwell,j,NO,2,FALSO,https://www.scopus.com/sourceid/12167,1739565,14390485
2,12274,Progress in Aerospace Sciences,1127,122,"81,1","9,2","3,675","1,433",2210,Mechanical Engineering,97,12,546,Elsevier,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/12274,3760421,NaN
3,12274,Progress in Aerospace Sciences,1127,122,"81,1","9,2","3,675","1,433",2202,Aerospace Engineering,99,1,110,Elsevier,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/12274,3760421,NaN
4,12274,Progress in Aerospace Sciences,1127,122,"81,1","9,2","3,675","1,433",2211,Mechanics of Materials,97,9,329,Elsevier,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/12274,3760421,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49627,12157,Journal of Electron Spectroscopy and Related P...,1597,504,71,"3,2","0,73","0,852",1607,Spectroscopy,53,31,65,Elsevier,j,NO,2,FALSO,https://www.scopus.com/sourceid/12157,3682048,NaN
49628,12157,Journal of Electron Spectroscopy and Related P...,1597,504,71,"3,2","0,73","0,852",3108,Radiation,64,17,46,Elsevier,j,NO,2,FALSO,https://www.scopus.com/sourceid/12157,3682048,NaN
49629,12157,Journal of Electron Spectroscopy and Related P...,1597,504,71,"3,2","0,73","0,852",3107,"Atomic and Molecular Physics, and Optics",64,59,163,Elsevier,j,NO,2,FALSO,https://www.scopus.com/sourceid/12157,3682048,NaN
49630,12157,Journal of Electron Spectroscopy and Related P...,1597,504,71,"3,2","0,73","0,852",2504,"Electronic, Optical and Magnetic Materials",65,77,219,Elsevier,j,NO,2,FALSO,https://www.scopus.com/sourceid/12157,3682048,NaN


In [ ]:
CS2015


,ScopusSourceID,Title,CitationCount,ScholarlyOutput,PercentCited,CiteScore,SNIP,SJR,ScopusASJCCode,ScopusSubSubjectArea,Percentile,RANK,RankOutOf,Publisher,Type,OpenAccess,Quartile,Top10,URLScopusSourceID,PrintISSN,ElectISSN
0,18619,Practical Neurology,510,260,"47,7",2,"0,694","0,438",2700,General Medicine,40,1948,3276,BMJ Publishing Group,j,NO,3,FALSO,https://www.scopus.com/sourceid/18619,14747758,14747766
1,18619,Practical Neurology,510,260,"47,7",2,"0,694","0,438",2728,Neurology (clinical),40,194,323,BMJ Publishing Group,j,NO,3,FALSO,https://www.scopus.com/sourceid/18619,14747758,14747766
2,19057,Tissue and Cell,570,235,"65,5","2,4","0,611","0,543",1309,Developmental Biology,19,62,76,Elsevier,j,NO,4,FALSO,https://www.scopus.com/sourceid/19057,408166,NaN
3,19057,Tissue and Cell,570,235,"65,5","2,4","0,611","0,543",1307,Cell Biology,20,208,261,Elsevier,j,NO,4,FALSO,https://www.scopus.com/sourceid/19057,408166,NaN
4,19235,Journal of Plant Nutrition,686,636,"44,8","1,1","0,788","0,435",1314,Physiology,11,149,167,Taylor & Francis,j,NO,4,FALSO,https://www.scopus.com/sourceid/19235,1904167,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50477,17409,Poetics,324,132,"64,4","2,5","1,451","0,986",3310,Linguistics and Language,89,69,631,Elsevier,j,NO,1,FALSO,https://www.scopus.com/sourceid/17409,0304422X,NaN
50478,17409,Poetics,324,132,"64,4","2,5","1,451","0,986",1208,Literature and Literary Theory,99,4,617,Elsevier,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/17409,0304422X,NaN
50479,17409,Poetics,324,132,"64,4","2,5","1,451","0,986",3315,Communication,83,42,252,Elsevier,j,NO,1,FALSO,https://www.scopus.com/sourceid/17409,0304422X,NaN
50480,17479,International Journal of Water Resources Devel...,531,188,"74,5","2,8","0,96","0,64",3303,Development,85,28,188,Taylor & Francis,j,NO,1,FALSO,https://www.scopus.com/sourceid/17479,7900627,13600648


In [ ]:
CS2014


,ScopusSourceID,Title,CitationCount,ScholarlyOutput,PercentCited,CiteScore,SNIP,SJR,ScopusASJCCode,ScopusSubSubjectArea,Percentile,RANK,RankOutOf,Publisher,Type,OpenAccess,Quartile,Top10,URLScopusSourceID,PrintISSN,ElectISSN
0,26775,Acta Oceanologica Sinica,369,489,"40,9","0,8","0,534","0,286",1910,Oceanography,20,88,110,Springer Nature,j,NO,4,FALSO,https://www.scopus.com/sourceid/26775,0253505X,18691099
1,26775,Acta Oceanologica Sinica,369,489,"40,9","0,8","0,534","0,286",1104,Aquatic Science,16,158,189,Springer Nature,j,NO,4,FALSO,https://www.scopus.com/sourceid/26775,0253505X,18691099
2,26847,Women and Criminal Justice,61,61,"45,9",1,"0,89","0,463",3318,Gender Studies,55,51,114,Taylor & Francis,j,NO,2,FALSO,https://www.scopus.com/sourceid/26847,8974454,NaN
3,26847,Women and Criminal Justice,61,61,"45,9",1,"0,89","0,463",3308,Law,54,207,461,Taylor & Francis,j,NO,2,FALSO,https://www.scopus.com/sourceid/26847,8974454,NaN
4,27514,Antioxidants and Redox Signaling,16875,1278,"91,3","13,2","1,742","3,137",1312,Molecular Biology,93,24,352,Mary Ann Liebert,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/27514,15230864,15577716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49291,26225,Journal of the Massachusetts Dental Society,21,199,"6,5","0,1",NaN,"0,118",2700,General Medicine,7,3026,3264,Massachusetts Dental Society,j,NO,4,FALSO,https://www.scopus.com/sourceid/26225,254800,NaN
49292,26485,Russian Journal of Organic Chemistry,1081,1213,"42,5","0,9","0,613","0,282",1605,Organic Chemistry,12,143,162,Pleiades Publishing,j,NO,4,FALSO,https://www.scopus.com/sourceid/26485,10704280,16083393
49293,26677,International Journal of Energy Technology and...,26,75,24,"0,3","0,22","0,152",2103,Fuel Technology,26,59,80,Inderscience,j,NO,3,FALSO,https://www.scopus.com/sourceid/26677,14728923,1741508X
49294,26677,International Journal of Energy Technology and...,26,75,24,"0,3","0,22","0,152",2102,Energy Engineering and Power Technology,26,129,175,Inderscience,j,NO,3,FALSO,https://www.scopus.com/sourceid/26677,14728923,1741508X


In [ ]:
CS2013


,ScopusSourceID,Title,CitationCount,ScholarlyOutput,PercentCited,CiteScore,SNIP,SJR,ScopusASJCCode,ScopusSubSubjectArea,Percentile,RANK,RankOutOf,Publisher,Type,OpenAccess,Quartile,Top10,URLScopusSourceID,PrintISSN,ElectISSN
0,12037,"Welding in the World, Le Soudage Dans Le Monde",255,278,36,"0,9","0,724","0,39",2211,Mechanics of Materials,42,179,309,Springer Nature,j,NO,3,FALSO,https://www.scopus.com/sourceid/12037,432288,NaN
1,12037,"Welding in the World, Le Soudage Dans Le Monde",255,278,36,"0,9","0,724","0,39",2210,Mechanical Engineering,46,274,510,Springer Nature,j,NO,3,FALSO,https://www.scopus.com/sourceid/12037,432288,NaN
2,12037,"Welding in the World, Le Soudage Dans Le Monde",255,278,36,"0,9","0,724","0,39",2506,Metals and Alloys,56,60,136,Springer Nature,j,NO,2,FALSO,https://www.scopus.com/sourceid/12037,432288,NaN
3,12127,Behaviour Research and Therapy,3711,496,"80,8","7,5","1,816","2,294",1201,Arts and Humanities (miscellaneous),96,11,267,Elsevier,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/12127,57967,NaN
4,12127,Behaviour Research and Therapy,3711,496,"80,8","7,5","1,816","2,294",2738,Psychiatry and Mental Health,93,32,463,Elsevier,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/12127,57967,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48134,21100264891,Applications of Management Science,7,13,"23,1","0,5",0,NaN,1400,"General Business,Management and Accounting",42,106,182,Emerald,k,NO,3,FALSO,https://www.scopus.com/sourceid/21100264891,2768976,NaN
48135,21100286446,Seizieme Siecle,0,13,0,0,0,"0,101",3316,Cultural Studies,4,586,636,Librairie Droz SA,j,YES,4,FALSO,https://www.scopus.com/sourceid/21100286446,17753856,17744466
48136,21100286446,Seizieme Siecle,0,13,0,0,0,"0,101",1202,History,3,756,815,Librairie Droz SA,j,YES,4,FALSO,https://www.scopus.com/sourceid/21100286446,17753856,17744466
48137,21100286446,Seizieme Siecle,0,13,0,0,0,"0,101",1213,Visual Arts and Performing Arts,6,299,344,Librairie Droz SA,j,YES,4,FALSO,https://www.scopus.com/sourceid/21100286446,17753856,17744466


In [ ]:
CS2012


,ScopusSourceID,Title,CitationCount,ScholarlyOutput,PercentCited,CiteScore,SNIP,SJR,ScopusASJCCode,ScopusSubSubjectArea,Percentile,RANK,RankOutOf,Publisher,Type,OpenAccess,Quartile,Top10,URLScopusSourceID,PrintISSN,ElectISSN
0,15205,Memorias do Instituto Oswaldo Cruz,3106,812,"66,6","3,8","0,93","0,728",2726,Microbiology (medical),64,35,96,Fundacao Oswaldo Cruz,j,YES,2,FALSO,https://www.scopus.com/sourceid/15205,740276,16788060
1,15505,Journal of Family Psychotherapy,72,89,"44,9","0,8","0,416","0,252",2738,Psychiatry and Mental Health,32,305,454,Taylor & Francis,j,NO,3,FALSO,https://www.scopus.com/sourceid/15505,8975353,15404080
2,15505,Journal of Family Psychotherapy,72,89,"44,9","0,8","0,416","0,252",3203,Clinical Psychology,34,155,236,Taylor & Francis,j,NO,3,FALSO,https://www.scopus.com/sourceid/15505,8975353,15404080
3,15557,American Political Science Review,965,171,"73,7","5,6","4,103","8,772",3312,Sociology and Political Science,98,14,855,Cambridge University Press,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/15557,30554,15375943
4,15673,Journal of Rural Studies,631,162,"77,2","3,9","2,008","1,189",3303,Development,94,11,179,Elsevier,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/15673,7430167,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46429,14997,International Agricultural Engineering Journal,40,112,"15,2","0,4","0,19","0,151",2210,Mechanical Engineering,23,384,502,Asian Association For Agricultural Engineering,j,NO,4,FALSO,https://www.scopus.com/sourceid/14997,8582114,NaN
46430,14997,International Agricultural Engineering Journal,40,112,"15,2","0,4","0,19","0,151",1100,General Agricultural and Biological Sciences,29,108,152,Asian Association For Agricultural Engineering,j,NO,3,FALSO,https://www.scopus.com/sourceid/14997,8582114,NaN
46431,15106,Journal of Inherited Metabolic Disease,2405,449,"80,2","5,4","1,114","0,845",1311,Genetics,71,83,287,Springer Nature,j,NO,2,FALSO,https://www.scopus.com/sourceid/15106,1418955,15732665
46432,15106,Journal of Inherited Metabolic Disease,2405,449,"80,2","5,4","1,114","0,845",2716,Genetics (clinical),62,33,87,Springer Nature,j,NO,2,FALSO,https://www.scopus.com/sourceid/15106,1418955,15732665


In [ ]:
CS2011

,ScopusSourceID,Title,CitationCount,ScholarlyOutput,PercentCited,CiteScore,SNIP,SJR,ScopusASJCCode,ScopusSubSubjectArea,Percentile,RANK,RankOutOf,Publisher,Type,OpenAccess,Quartile,Top10,URLScopusSourceID,PrintISSN,ElectISSN
0,12253,China Business Review,43,160,"18,8","0,3","0,074","0,135",1403,Business and International Management,27,204,281,National Council for US-China Trade,j,NO,3,FALSO,https://www.scopus.com/sourceid/12253,1637169,NaN
1,12253,China Business Review,43,160,"18,8","0,3","0,074","0,135",1408,Strategy and Management,16,247,295,National Council for US-China Trade,j,NO,4,FALSO,https://www.scopus.com/sourceid/12253,1637169,NaN
2,12253,China Business Review,43,160,"18,8","0,3","0,074","0,135",3320,Political Science and International Relations,27,253,350,National Council for US-China Trade,j,NO,3,FALSO,https://www.scopus.com/sourceid/12253,1637169,NaN
3,12253,China Business Review,43,160,"18,8","0,3","0,074","0,135",2001,"Economics, Econometrics and Finance (miscellan...",19,45,55,National Council for US-China Trade,j,NO,4,FALSO,https://www.scopus.com/sourceid/12253,1637169,NaN
4,12881,Behavioral and Social Sciences Librarian,39,83,"24,1","0,5","0,401","0,258",3300,General Social Sciences,43,93,165,Taylor & Francis,j,NO,3,FALSO,https://www.scopus.com/sourceid/12881,1639269,15444546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44430,12085,American Archivist,99,85,"37,6","1,2","1,891","0,605",1201,Arts and Humanities (miscellaneous),46,141,264,Society of American Archivists,j,NO,3,FALSO,https://www.scopus.com/sourceid/12085,3609081,NaN
44431,12216,Molecular Biology and Evolution,10939,1077,"89,7","10,2","1,854","4,257",1312,Molecular Biology,88,39,335,Oxford University Press,j,NO,1,FALSO,https://www.scopus.com/sourceid/12216,7374038,15371719
44432,12216,Molecular Biology and Evolution,10939,1077,"89,7","10,2","1,854","4,257",1105,"Ecology, Evolution, Behavior and Systematics",97,12,478,Oxford University Press,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/12216,7374038,15371719
44433,12216,Molecular Biology and Evolution,10939,1077,"89,7","10,2","1,854","4,257",2700,General Medicine,94,177,3161,Oxford University Press,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/12216,7374038,15371719


# Nueva sección